In [7]:
import pandas as pd
import numpy as np
import os
def normalize_global(df, indicator_cols):
    min_vals = df[indicator_cols].min()
    max_vals = df[indicator_cols].max()
    return (df[indicator_cols] - min_vals) / (max_vals - min_vals + 1e-10)

def compute_entropy_weights(df, indicator_cols):
    norm_df = normalize_global(df, indicator_cols).replace(0, 1e-10)
    P = norm_df.div(norm_df.sum(axis=0), axis=1)
    logP = np.log(P)
    E = -(P * logP).sum(axis=0) / np.log(len(df))
    d = 1 - E
    w = d / d.sum()
    return w

def compute_index(df, indicator_cols, weights):
    norm_df = normalize_global(df, indicator_cols)
    return norm_df.dot(weights)

# ==== Step 1：读取所有城市数据 ====
citys = ["福州",
        "莆田",
        "三明",
        "龙岩",
        "厦门",
        "泉州",
        "漳州",
        "宁德",
        "南平"]
dir_path = "E:\Code\数字经济指标\分城市合并"
indicator_cols = ['专利','招聘','云计算与AI', '产业数字化', '数字化治理', '数据价值化']
all_data = []

for city in citys:
    city_df = pd.read_excel(os.path.join(dir_path,f"{city}数字经济指标汇总2022-2024.xlsx"))
    city_df['city'] = city  # 添加城市列
    all_data.append(city_df)

# 拼接所有城市数据
df_all = pd.concat(all_data, ignore_index=True)

# ==== Step 2：选定统一基准期数据计算熵权 ====
base_df = df_all[df_all['week'].between('2022-01', '2022-52')]
weights_entropy = compute_entropy_weights(base_df, indicator_cols)

# ==== Step 3：融合主观权重（可选）====
manual_weights = pd.Series({'专利': 0.20, '招聘':0.10, '云计算与AI': 0.20,
                             '产业数字化': 0.30, '数字化治理': 0.10, '数据价值化': 0.10})
# final_weights = weights_entropy * manual_weights
# final_weights = final_weights / final_weights.sum()  # 权重归一化
# print("最终权重：\n", final_weights)

# ==== Step 4：统一计算指数 ====
df_all['数字经济指数'] = compute_index(df_all, indicator_cols, manual_weights)
print("最终权重：\n", manual_weights)
# ==== Step 5：导出或分城市分析 ====
df_all.to_excel("所有城市_数字经济指数_可比1.xlsx", index=False)

# 示例：按城市和周查看趋势
pivot = df_all.pivot(index='week', columns='city', values='数字经济指数')
print(pivot.head())

最终权重：
 专利        0.2
招聘        0.1
云计算与AI    0.2
产业数字化     0.3
数字化治理     0.1
数据价值化     0.1
dtype: float64
city           三明        南平        厦门        宁德        泉州        漳州        福州  \
week                                                                            
2021-52  0.010738  0.007893  0.070686  0.012341  0.056837  0.023402  0.078054   
2022-01  0.113946  0.113657  0.465714  0.139222  0.325156  0.201506  0.520823   
2022-02  0.110855  0.112297  0.497871  0.120683  0.338873  0.211899  0.555158   
2022-03  0.108000  0.101276  0.522170  0.132270  0.344877  0.192064  0.517452   
2022-04  0.130124  0.123387  0.561204  0.142049  0.339258  0.206361  0.557650   

city           莆田        龙岩  
week                         
2021-52  0.028620  0.001597  
2022-01  0.162436  0.112418  
2022-02  0.184317  0.110075  
2022-03  0.189889  0.130256  
2022-04  0.187408  0.126551  


In [8]:
pivot.to_excel("福建省数字经济指数1.xlsx")